In [ ]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import json
import os
import joblib
from datetime import datetime
import warnings
import logging

# 2. 서드파티 라이브러리 
import numpy as np
import pandas as pd

# 2-1. 시각화
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR, REPORT_DIR

IMG_DIR = REPORT_DIR / 'figures'

In [ ]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [ ]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

In [ ]:
df = milk[milk['가격미달'] == 0]
df.shape

In [ ]:
features = [
    '전산차건유전유량', 
    '산차', 
    '농후사료급여량(연계)', 
    '공태일수', 
    '계절', 
    '농장구분', 
    '분만간격',
    # '정액코드분류'
]

target = '가격'

In [ ]:
warnings.filterwarnings('ignore')

# ============================================
# 0. 저장 디렉토리 및 타임스탬프 설정
# ============================================

# 현재 시각 측정 (모든 파일에 동일하게 사용)
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

# 모든 파일 경로 미리 정의
PATHS = {
    'log': MODEL_DIR / f'optuna_xgboost_{TIMESTAMP}.log',
    'db': f'sqlite:///{MODEL_DIR / f"optuna_xgboost_{TIMESTAMP}.db"}',
    'intermediate': MODEL_DIR / f'intermediate_results_{TIMESTAMP}.json',
    'best_params': MODEL_DIR / f'best_params_{TIMESTAMP}.json',
    'all_trials': MODEL_DIR / f'all_trials_{TIMESTAMP}.csv',
    'test_results': MODEL_DIR / f'test_results_{TIMESTAMP}.json',
    'predictions': MODEL_DIR / f'test_predictions_{TIMESTAMP}.csv',
    'feature_importance': MODEL_DIR / f'feature_importance_{TIMESTAMP}.csv',
    'final_model': MODEL_DIR / f'xgboost_final_model_{TIMESTAMP}.pkl',
    'final_report': MODEL_DIR / f'final_report_{TIMESTAMP}.txt',
    # 시각화 파일들
    'viz_history': IMG_DIR / f'optuna_optimization_history_{TIMESTAMP}.html',
    'viz_importance': IMG_DIR / f'optuna_param_importances_{TIMESTAMP}.html',
    'viz_contour': IMG_DIR / f'optuna_contour_{TIMESTAMP}.html',
    'viz_parallel': IMG_DIR / f'optuna_parallel_coordinate_{TIMESTAMP}.html',
    'viz_slice': IMG_DIR / f'optuna_slice_{TIMESTAMP}.html',
}

# ============================================
# 1. 로깅 설정
# ============================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(PATHS['log']),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

logger.info(f"📁 저장 디렉토리: {MODEL_DIR.absolute()}")
logger.info(f"⏰ 타임스탬프: {TIMESTAMP}")

# ============================================
# 2. 멀티 GPU 설정
# ============================================
AVAILABLE_GPUS = [0, 1, 2, 3]  # 당신의 GPU 개수에 맞게 수정
NUM_GPUS = len(AVAILABLE_GPUS)

logger.info(f"🎮 사용 가능한 GPU: {AVAILABLE_GPUS}")

def get_gpu_id(trial_number):
    """Trial 번호에 따라 GPU 할당 (라운드 로빈 방식)"""
    return AVAILABLE_GPUS[trial_number % NUM_GPUS]

# ============================================
# 3. 데이터 준비
# ============================================
logger.info("📁 데이터 로딩 중...")

split_date = '2021-08-01'
train = df[df['검정일자'] < split_date]
test = df[df['검정일자'] >= split_date]

X_train, X_test = train[features], test[features]
y_train, y_test = train[target], test[target]

logger.info(f"  훈련 데이터: {X_train.shape}")
logger.info(f"  테스트 데이터: {X_test.shape}")

# ============================================
# 4. 중간 결과 저장 함수 (optimize 전에 정의 필요)
# ============================================
def save_intermediate_results(study, trial):
    """10번마다 중간 결과 저장"""
    logger.info(f"\n💾 중간 저장 (Trial {trial.number})")
    
    intermediate_results = {
        'timestamp': TIMESTAMP,
        'saved_at': datetime.now().isoformat(),
        'trial_number': trial.number,
        'best_value': study.best_value,
        'best_params': study.best_params,
        'n_trials': len(study.trials)
    }
    
    with open(PATHS['intermediate'], 'w') as f:
        json.dump(intermediate_results, f, indent=2)

# ============================================
# 5. Objective 함수 (멀티 GPU 버전)
# ============================================
def objective(trial):
    """
    Optuna 목적 함수 - 멀티 GPU 지원
    """
    # 5-1. GPU 할당
    gpu_id = get_gpu_id(trial.number)
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
    
    logger.info(f"Trial {trial.number}: GPU {gpu_id} 사용")
    
    # 5-2. 파라미터 제안
    params = {
        # ========== 모델 복잡도 ==========
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        
        # ========== 학습 속도 & 반복 ==========
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
        
        # ========== 샘플링 ==========
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        
        # ========== 정규화 ==========
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        
        # ========== GPU & 기타 ==========
        'tree_method': 'gpu_hist',
        'gpu_id': gpu_id,
        'predictor': 'gpu_predictor',
        'random_state': 42,
        'verbosity': 0
    }
    
    # 5-3. 모델 생성 및 교차 검증
    try:
        model = XGBRegressor(**params)
        
        cv_scores = cross_val_score(
            model, X_train, y_train,
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=1
        )
        
        mean_mse = -cv_scores.mean()
        
        # Trial에 추가 정보 저장
        trial.set_user_attr('cv_std', cv_scores.std())
        trial.set_user_attr('cv_scores', cv_scores.tolist())
        trial.set_user_attr('gpu_id', gpu_id)
        
        return mean_mse
        
    except Exception as e:
        logger.error(f"Trial {trial.number} 실패: {str(e)}")
        raise optuna.exceptions.TrialPruned()

# ============================================
# 6. Optuna Study 생성 (DB 저장)
# ============================================
STUDY_NAME = f'xgboost_regression_{TIMESTAMP}'

logger.info(f"🗄️  데이터베이스: {PATHS['db']}")
logger.info(f"📚 Study 이름: {STUDY_NAME}")

# Study 생성 또는 불러오기
study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=PATHS['db'],
    load_if_exists=True,
    direction='minimize',
    sampler=optuna.samplers.TPESampler(
        seed=42,
        n_startup_trials=20,
        multivariate=True
    ),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=10,
        n_warmup_steps=5
    )
)

# 이전 실행 정보 출력
if len(study.trials) > 0:
    logger.info(f"✅ 이전 실행 발견: {len(study.trials)}개 trials")
    logger.info(f"   현재 최적 MSE: {study.best_value:.4f}")

# ============================================
# 7. 최적화 실행
# ============================================
N_TRIALS = 300
TIMEOUT = None

logger.info("\n" + "="*60)
logger.info("🚀 Optuna 최적화 시작!")
logger.info(f"   목표 Trials: {N_TRIALS}")
logger.info(f"   멀티 GPU: {NUM_GPUS}개")
logger.info("="*60 + "\n")

study.optimize(
    objective,
    n_trials=N_TRIALS,
    timeout=TIMEOUT,
    n_jobs=1,
    show_progress_bar=True,
    callbacks=[
        lambda study, trial: save_intermediate_results(study, trial) 
        if trial.number % 10 == 0 else None
    ]
)

# ============================================
# 8. 최적화 결과 분석 및 저장
# ============================================
logger.info("\n" + "="*60)
logger.info("✅ 최적화 완료!")
logger.info("="*60)

# 8-1. 기본 통계
logger.info(f"\n📊 탐색 통계:")
logger.info(f"  총 시도: {len(study.trials)}")
logger.info(f"  완료: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
logger.info(f"  가지치기: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
logger.info(f"  실패: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")

# 8-2. 최적 파라미터
logger.info(f"\n🏆 최적 파라미터:")
for key, value in study.best_params.items():
    logger.info(f"  {key:20s}: {value}")

logger.info(f"\n🎯 최적 CV MSE: {study.best_value:.6f}")

# ============================================
# 9. 최적 파라미터 저장 (JSON)
# ============================================
best_params_with_meta = {
    'timestamp': TIMESTAMP,
    'study_name': STUDY_NAME,
    'n_trials': len(study.trials),
    'best_value': study.best_value,
    'best_params': study.best_params,
    'cv_std': study.best_trial.user_attrs.get('cv_std', None),
    'gpu_id_used': study.best_trial.user_attrs.get('gpu_id', None)
}

with open(PATHS['best_params'], 'w') as f:
    json.dump(best_params_with_meta, f, indent=2)

logger.info(f"\n💾 저장: {PATHS['best_params'].name}")

# ============================================
# 10. 전체 Trials 정보 저장 (CSV)
# ============================================
trials_df = study.trials_dataframe()
trials_df.to_csv(PATHS['all_trials'], index=False)
logger.info(f"💾 저장: {PATHS['all_trials'].name}")

# ============================================
# 11. 최종 모델 학습
# ============================================
logger.info("\n" + "="*60)
logger.info("🔧 최종 모델 학습 중...")
logger.info("="*60)

os.environ['CUDA_VISIBLE_DEVICES'] = str(AVAILABLE_GPUS[0])

best_params_final = study.best_params.copy()
best_params_final.update({
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
    'predictor': 'gpu_predictor',
    'random_state': 42,
    'verbosity': 1
})

final_model = XGBRegressor(**best_params_final)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric=['rmse', 'mae'],
    early_stopping_rounds=50,
    verbose=50
)

# ============================================
# 12. 테스트 성능 평가
# ============================================
logger.info("\n" + "="*60)
logger.info("📊 테스트 세트 평가")
logger.info("="*60)

y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

# 훈련 세트 성능
train_mse = mean_squared_error(y_train, y_pred_train)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)

# 테스트 세트 성능
test_mse = mean_squared_error(y_test, y_pred_test)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)

logger.info(f"\n🎓 훈련 세트:")
logger.info(f"  MSE  : {train_mse:.6f}")
logger.info(f"  RMSE : {train_rmse:.6f}")
logger.info(f"  MAE  : {train_mae:.6f}")
logger.info(f"  R²   : {train_r2:.6f}")

logger.info(f"\n🎯 테스트 세트:")
logger.info(f"  MSE  : {test_mse:.6f}")
logger.info(f"  RMSE : {test_rmse:.6f}")
logger.info(f"  MAE  : {test_mae:.6f}")
logger.info(f"  R²   : {test_r2:.6f}")

# 과적합 체크
overfitting_ratio = (train_mse - test_mse) / train_mse * 100
logger.info(f"\n⚠️  과적합 지표: {overfitting_ratio:.2f}%")
if overfitting_ratio > 20:
    logger.warning("  → 과적합 의심! 정규화 강화 고려")
elif overfitting_ratio < -10:
    logger.warning("  → 과소적합 의심! 모델 복잡도 증가 고려")
else:
    logger.info("  → 적절한 일반화 성능 ✅")

# ============================================
# 13. 테스트 결과 저장 (JSON)
# ============================================
test_results = {
    'timestamp': TIMESTAMP,
    'model': 'XGBRegressor',
    'optimization': 'Optuna',
    'n_trials': len(study.trials),
    'best_cv_mse': study.best_value,
    
    'train_metrics': {
        'mse': float(train_mse),
        'rmse': float(train_rmse),
        'mae': float(train_mae),
        'r2': float(train_r2)
    },
    
    'test_metrics': {
        'mse': float(test_mse),
        'rmse': float(test_rmse),
        'mae': float(test_mae),
        'r2': float(test_r2)
    },
    
    'overfitting_ratio': float(overfitting_ratio),
    'best_params': study.best_params,
    
    'training_info': {
        'train_size': len(X_train),
        'test_size': len(X_test),
        'n_features': X_train.shape[1],
        'early_stopping_rounds': 50,
        'best_iteration': int(final_model.best_iteration) if hasattr(final_model, 'best_iteration') else None
    }
}

with open(PATHS['test_results'], 'w') as f:
    json.dump(test_results, f, indent=2)

logger.info(f"\n💾 저장: {PATHS['test_results'].name}")

# ============================================
# 14. 예측 결과 저장 (CSV)
# ============================================
predictions_df = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_pred_test,
    'error': y_test - y_pred_test,
    'abs_error': np.abs(y_test - y_pred_test),
    'squared_error': (y_test - y_pred_test) ** 2
})

predictions_df.to_csv(PATHS['predictions'], index=False)
logger.info(f"💾 저장: {PATHS['predictions'].name}")

# ============================================
# 15. Feature Importance 저장
# ============================================
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance.to_csv(PATHS['feature_importance'], index=False)
logger.info(f"💾 저장: {PATHS['feature_importance'].name}")

logger.info("\n🔍 Top 10 중요 Features:")
for idx, row in feature_importance.head(10).iterrows():
    logger.info(f"  {row['feature']:20s}: {row['importance']:.4f}")

# ============================================
# 16. 최종 모델 저장 (Pickle)
# ============================================
joblib.dump(final_model, PATHS['final_model'])
logger.info(f"\n💾 저장: {PATHS['final_model'].name}")

# ============================================
# 17. Optuna 시각화
# ============================================
logger.info("\n" + "="*60)
logger.info("📊 시각화 생성 중...")
logger.info("="*60)

try:
    # 17-1. 최적화 과정
    fig1 = optuna.visualization.plot_optimization_history(study)
    fig1.write_html(str(PATHS['viz_history']))
    logger.info(f"✅ {PATHS['viz_history'].name}")
    
    # 17-2. 파라미터 중요도
    fig2 = optuna.visualization.plot_param_importances(study)
    fig2.write_html(str(PATHS['viz_importance']))
    logger.info(f"✅ {PATHS['viz_importance'].name}")
    
    # 17-3. 파라미터 관계도
    fig3 = optuna.visualization.plot_contour(study, params=['learning_rate', 'max_depth'])
    fig3.write_html(str(PATHS['viz_contour']))
    logger.info(f"✅ {PATHS['viz_contour'].name}")
    
    # 17-4. 병렬 좌표 플롯
    fig4 = optuna.visualization.plot_parallel_coordinate(study)
    fig4.write_html(str(PATHS['viz_parallel']))
    logger.info(f"✅ {PATHS['viz_parallel'].name}")
    
    # 17-5. Slice 플롯
    fig5 = optuna.visualization.plot_slice(study)
    fig5.write_html(str(PATHS['viz_slice']))
    logger.info(f"✅ {PATHS['viz_slice'].name}")
    
except Exception as e:
    logger.warning(f"시각화 생성 중 오류: {str(e)}")

# ============================================
# 18. 최종 요약 리포트 저장
# ============================================
summary_report = f"""
{'='*60}
XGBoost + Optuna 최적화 최종 리포트
{'='*60}

📅 실행 시간: {TIMESTAMP}
📁 저장 위치: {MODEL_DIR.absolute()}

🎮 GPU 설정:
  - 사용 GPU: {AVAILABLE_GPUS}
  - GPU 개수: {NUM_GPUS}

🔍 최적화 설정:
  - Study 이름: {STUDY_NAME}
  - 총 Trials: {len(study.trials)}
  - 완료된 Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}

🏆 최적 파라미터:
{json.dumps(study.best_params, indent=2)}

📊 성능 지표:
  
  훈련 세트:
    - MSE  : {train_mse:.6f}
    - RMSE : {train_rmse:.6f}
    - MAE  : {train_mae:.6f}
    - R²   : {train_r2:.6f}
  
  테스트 세트:
    - MSE  : {test_mse:.6f}
    - RMSE : {test_rmse:.6f}
    - MAE  : {test_mae:.6f}
    - R²   : {test_r2:.6f}
  
  과적합 비율: {overfitting_ratio:.2f}%

💾 저장된 파일:
  - {PATHS['best_params'].name}
  - {PATHS['test_results'].name}
  - {PATHS['all_trials'].name}
  - {PATHS['predictions'].name}
  - {PATHS['feature_importance'].name}
  - {PATHS['final_model'].name}
  - {PATHS['db'].split('///')[-1]}
  - {PATHS['log'].name}
  - {PATHS['viz_history'].name}
  - {PATHS['viz_importance'].name}
  - {PATHS['viz_contour'].name}
  - {PATHS['viz_parallel'].name}
  - {PATHS['viz_slice'].name}

{'='*60}
"""

with open(PATHS['final_report'], 'w', encoding='utf-8') as f:
    f.write(summary_report)

logger.info(f"\n💾 저장: {PATHS['final_report'].name}")
logger.info(summary_report)

logger.info("\n" + "="*60)
logger.info("🎉 모든 작업 완료!")
logger.info(f"📁 모든 파일이 {MODEL_DIR.absolute()} 에 저장되었습니다")
logger.info("="*60)